In [1]:
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.preprocessing.image 

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12799989710589843628
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2258055988
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6414287963929279499
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
path = "E:\deep learning project\inaturalist_12K"

In [5]:
def get_data(path,augmentation=True):
    train_path=os.path.join(path,"train")
    test_path=os.path.join(path,"val")
    if augmentation==False:
        train_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.1)
    else:
        train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                          rotation_range=90,
                                          zoom_range=0.2,
                                          shear_range=0.2,
                                          validation_split=0.1,
                                          horizontal_flip=True)
    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    train_data = train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    valid_data=train_generator.flow_from_directory(
    directory=train_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    test_data=test_generator.flow_from_directory(
    directory=test_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42)
    return train_data,valid_data,test_data

In [6]:
image_size=256
output_size=10

In [ ]:
import o

In [7]:
import wandb

In [9]:
wandb.login()

True

In [10]:
from wandb.keras import WandbCallback
class CNN(object):
    def __init__(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
               no_filters=32,image_size=256,drop_out=0.2,output_size=10,
               augmentation= True,batch_normalization=True):
        self.kenel_size=kernel_size
        self.learning_rate=learning_rate
        self.image_size=image_size
        self.output_size=output_size
        self.augmentation=augmentation
        self.batch_normalization=batch_normalization
        self.no_conv_layers=no_conv_layers
        self.dense_layer_size=dense_layer_size
        self.drop_out=drop_out
        self.no_filters=no_filters
        self.epochs=epochs
        self.padding=padding
        self.filter_no_metric= filter_no_metric
        self.activation_func=activation_func
        self.initialize(no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size,activation_func,
                    no_filters,image_size,drop_out,output_size,
                    augmentation,batch_normalization)
    def initialize(self,no_conv_layers,kernel_size,learning_rate,epochs,padding,filter_no_metric,dense_layer_size=32,activation_func='relu',
                 no_filters=32,image_size=256,drop_out=0.2,output_size=10,
                    augmentation= True,batch_normalization=True
                 ):
        self.model=Sequential()
        print(activation_func)
        for i in range(0,no_conv_layers):
            if i==0:
                self.model.add(Conv2D(no_filters,kernel_size, input_shape=(image_size, image_size, 3),kernel_initializer = "he_uniform",padding = padding,
                           data_format="channels_last"))
            else:
                if filter_no_metric=="1":
                    self.model.add(Conv2D(no_filters,kernel_size,kernel_initializer = "he_uniform",padding =padding))
                elif filter_no_metric=="2":
                    self.model.add(Conv2D(no_filters*(2**i),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))
                elif filter_no_metric=="1/2":
                    self.model.add(Conv2D(no_filters*(1/(2**i)),kernel_size,kernel_initializer = "he_uniform",
                                          padding =padding))

            self.model.add(Activation(activation_func))
            if batch_normalization==True:
                self.model.add(BatchNormalization())
            self.model.add(MaxPooling2D(pool_size=(2,2)))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Flatten())
        self.model.add(Dense(dense_layer_size))
        self.model.add(Activation(activation_func))
        if batch_normalization==True:
            self.model.add(BatchNormalization())
        self.model.add(Dropout(drop_out))
        self.model.add(Dense(output_size))
        self.model.add(Activation("softmax"))




In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
classes=["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
train_data,valid_data,test_data= get_data(path,True)
best_model=CNN(5,(3,3),0.001,1,'same','1',64,'elu',
              32,image_size,0.4,output_size,True,True)
best_model.model.compile(
optimizer=Adam(0.001),  # Optimizer
loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
best_model.model.fit(train_data,epochs=1,batch_size=32,validation_data=valid_data)
best_model.model.evaluate(test_data,batch_size=32)

In [ ]:
best_model.model.summary()

In [ ]:
!pip install matplotlib

In [ ]:
!pip install py-opencv

In [ ]:
test_path=os.path.join(path,"val")
img_path=os.path.join(test_path,"Amphibia")
lst=os.listdir(img_path)
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
try:
    image = cv2.imread(os.path.join(img_path,lst[0]))
    image=cv2.resize(image,(256,256))
    image=image/255
    plt.imshow(image)
    pred=best_model.model.predict(image)
    print(pred)
except:
    print("not _read")

In [ ]:
import matplotlib.pyplot as plt
def test(model,test_path,test_data,cls,classes):
    global k
    k=1
    img_path=os.path.join(test_path,cls)
    lst=os.listdir(img_path)
    i=0
    fig=plt.figure(figsize=(20,30))
    for img in lst:
        if i!=3:
            try:
                image = cv2.imread(os.path.join(img_path,img))
                image=cv2.resize(image,(image_size,image_size))
                fig.add_subplot(10,3,k)
                plt.imshow(image)
                plt.axis("off")
                image=image/255
                pred=model.predict(image.reshape(1,image_size,image_size,3))
                c=pred.argmax()
                plt.title("True_label-"+cls+"\n"+"pred_label-"+classes[c])
                i=i+1
                k=k+1
            except:
                break
        elif i==3:
            break

In [ ]:
for cls in classes:
    test(best_model.model,test_path,test_data,cls,classes)

# filter visualization

In [ ]:
best_model.model.evaluate(test_data, batch_size=256)

In [ ]:
from tensorflow.keras.models import Sequential, Model
import random
def plot_first_layer_filter(test_data,classes,model,layer='conv2d'):
    outputs =model.get_layer(layer).output
    inputs=model.inputs
    filter_maps_output=Model(inputs=inputs,outputs=outputs)
    length=len(classes)
    n_2=random.randrange(test_data[0][0].shape[0])
    n_3=random.randrange(test_data[0][0].shape[0])
    filter_1=filter_maps_output(test_data[n_2][0])
    plt.imshow(test_data[n_2][0][n_3])
    plt.axis("off")
    wandb.init(project="filter", entity="cs21m003_cs21d406")
    wandb.log({"true_image":plt})
    no_filters=filter_1.shape[3]
    figsize=(30,30)
    fig=plt.figure(figsize=figsize)
    k=1
    rows = 8
    columns=4
    for i in range(no_filters):
        fig.add_subplot(rows,columns,k)
        plt.imshow(filter_1[n_3,:,:,i])
        plt.title(str(i+1)+"_filter")
        plt.axis("off")
        k=k+1
    wandb.log({"filters":plt})
        
    

In [ ]:
plot_first_layer_filter(test_data,classes,best_model.model,'conv2d')

In [ ]:
best_model.model.summary()
outputs = [best_model.model.get_layer('conv2d').output]
inputs=best_model.model.inputs
from tensorflow.keras.models import Sequential, Model
fil_model = Model(
      inputs = inputs,    
      outputs = outputs )
print(test_data[2][0].shape)
feature_maps = fil_model(test_data[0][0])
plt.imshow(feature_maps[2,:,:,2])

# guided backpropagation

In [ ]:
def

# wand b process

In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
class Runner(object):
    def __init__(self,inaturalist_data=True,augmentation=True):
        if inaturalist_data:
            self.initialize_inaturalist_data(augmentation)
        else:
            self.initialize_data()

    def initialize_inaturalist_data(self,augmentation=True):
        self.train_data,self.valid_data,self.test_data= get_data(path,augmentation)
    

    def initialize_data(self):
        raise NotImplementedError("Please implement this method if you need other dataset.")

    @staticmethod
    def get_activation_function(key):
        mapper = {
      #"sigmoid": Sigmoid,
      "relu": 'relu'
      #"tanh": Tanh
    }
        assert key in mapper
        return mapper[key]

def run_wandb():
    wandb.init(project="bayes", entity="cs21m003_cs21d406")
    config = wandb.config
    wandb.run.name=f"e_{config.epochs}_bs_{config.batch_size}_kernel_size_{config.kernel_size}_filters_{config.no_filters}_ac_{config.act_func}_rate_{config.learning_rate}_aug_{config.augmentation}_BN_{config.batch_normalization}_drp_{config.drop_out}_pad_{config.padding}_dense_{config.dense_size}_metric_{config.filter_no_metric}_type_{config.type}"
    runner=Runner(True,config.augmentation)
    """
    params = {
    "epochs"        : config.epochs,
    "batch_size"    : config.batch_size,
    "kernel_size"   : config.kernel_size,
    "no_filters"    : config.no_filters,
    "act_func"      : config.act_func,
    "learning_rate" : config.learning_rate,
    "augmentation"  : config.augmentation,
    "batch_normalization"  : config.batch_normalization,
    "drop_out"  : config.drop_out,
    "padding": config.padding,
    "dense_size":config.dense_size,
    "filter_no_metric":config.filter_no_metric

    }
    """
  #no_conv_layers,filter_size,learning_rate,epochs,padding,filter_size_metric,dense_layer_size=32,activation_func='relu',
              # no_filters=32,image_size=256,drop_out=0.2,output_size=10,
               #augmentation= True,batch_normalization=True)
    model_1=CNN(5,config.kernel_size,config.learning_rate,config.epochs,config.padding,config.filter_no_metric,config.dense_size,config.act_func,
              config.no_filters,image_size,config.drop_out,output_size,config.augmentation,config.batch_normalization)
    model_1.model.compile(
    optimizer=Adam(config.learning_rate),  # Optimizer
    loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
    model_1.model.fit(runner.train_data,epochs=config.epochs,batch_size=config.batch_size,validation_data=runner.valid_data,
          callbacks=[WandbCallback()])
    loss,accuracy=model_1.model.evaluate(runner.test_data,batch_size=config.batch_size)
    print(f'test accuracy:{accuracy}')
    wandb.log({"test accuracy":accuracy})

def do_hyperparameter_search_using_wandb():
    sweep_config = {
    "name": "bayes sweep",
    "method": "bayes",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "type": {"values": ['bayes']},
      "epochs": {"values": [5, 10]}, 
      "batch_size": {"values": [64]}, 
      "kernel_size": {"values": [(3,3),(4,4),(5,5)]}, 
      "no_filters": {"values": [32,64]},
      "act_func": {"values": ['selu']}, 
      "learning_rate": {"values": [1e-3, 1e-4]}, 
      "augmentation": {"values": [True,False]} , 
      "batch_normalization": {"values": [True,False]},
      "drop_out": {"values": [0.4,0.5,0.6]},
      "padding": {"values": ['same','valid']},
      "dense_size": {"values": [32,64]},
      "filter_no_metric": {"values": ["1","2","1/2"]}}}
  
    sweep_id = wandb.sweep(sweep_config, project = "bayes",entity='cs21m003_cs21d406')
    wandb.agent(sweep_id, function=run_wandb,count=10)


if __name__ == '__main__':
    do_hyperparameter_search_using_wandb()  

Create sweep with ID: 9ki4kydb
Sweep URL: https://wandb.ai/cs21m003_cs21d406/bayes/sweeps/9ki4kydb


wandb: Agent Starting Run: jfm6jaeb with config:
wandb: 	act_func: selu
wandb: 	augmentation: False
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	drop_out: 0.5
wandb: 	epochs: 10
wandb: 	filter_no_metric: 1/2
wandb: 	kernel_size: [5, 5]
wandb: 	learning_rate: 0.0001
wandb: 	no_filters: 64
wandb: 	padding: same
wandb: 	type: bayes
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
selu
Epoch 1/10
313/313 [==============================] - 474s 1s/step - loss: 2.8797 - categorical_accuracy: 0.1212 - val_loss: 2.2361 - val_categorical_accuracy: 0.1811
Epoch 2/10
313/313 [==============================] - 470s 2s/step - loss: 2.3670 - categorical_accuracy: 0.1570 - val_loss: 2.1597 - val_categorical_accuracy: 0.2227
Epoch 3/10
313/313 [==============================] - 459s 1s/step - loss: 2.2771 - categorical_accuracy: 0.1780 - val_loss: 2.1178 - val_categorical_accuracy: 0.2348
Epoch 4/10
313/313 [==============================] - 438s 1s/step - loss: 2.2246 - categorical_accuracy: 0.1978 - val_loss: 2.0581 - val_categorical_accuracy: 0.2690
Epoch 5/10
313/313 [==============================] - 436s 1s/step - loss: 2.1846 - categorical_accuracy: 0.2114 - val_loss: 2.0625 - val_categorical_accuracy: 0.2529
Epoch 6/10
313/313 [===========

categorical_accuracy,▁▃▄▅▅▆▆▇▇█
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▂▁▁▁
test accuracy,▁
val_categorical_accuracy,▁▃▄▆▅▃▇██▇
val_loss,█▆▅▄▄▆▂▁▁▁
best_epoch,9
best_val_loss,1.95692
categorical_accuracy,0.26943
epoch,9
loss,2.05074


wandb: Agent Starting Run: py9vq4yb with config:
wandb: 	act_func: selu
wandb: 	augmentation: False
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	drop_out: 0.6
wandb: 	epochs: 5
wandb: 	filter_no_metric: 1/2
wandb: 	kernel_size: [5, 5]
wandb: 	learning_rate: 0.0001
wandb: 	no_filters: 64
wandb: 	padding: same
wandb: 	type: bayes
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

categorical_accuracy,▁▅▆██
epoch,▁▃▅▆█
loss,█▅▄▂▁
test accuracy,▁
val_categorical_accuracy,▁▄▆▆█
val_loss,█▆▃▃▁
best_epoch,4
best_val_loss,1.93205
categorical_accuracy,0.22652
epoch,4
loss,2.48254


wandb: Agent Starting Run: b0yuqp0h with config:
wandb: 	act_func: selu
wandb: 	augmentation: False
wandb: 	batch_normalization: True
wandb: 	batch_size: 64
wandb: 	dense_size: 64
wandb: 	drop_out: 0.6
wandb: 	epochs: 10
wandb: 	filter_no_metric: 1/2
wandb: 	kernel_size: [5, 5]
wandb: 	learning_rate: 0.001
wandb: 	no_filters: 64
wandb: 	padding: same
wandb: 	type: bayes
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
selu
Epoch 1/10



Run b0yuqp0h errored: ResourceExhaustedError()
wandb: ERROR Run b0yuqp0h errored: ResourceExhaustedError()
wandb: Agent Starting Run: onirf66p with config:
wandb: 	act_func: selu
wandb: 	augmentation: False
wandb: 	batch_normalization: False
wandb: 	batch_size: 64
wandb: 	dense_size: 32
wandb: 	drop_out: 0.5
wandb: 	epochs: 5
wandb: 	filter_no_metric: 1
wandb: 	kernel_size: [4, 4]
wandb: 	learning_rate: 0.0001
wandb: 	no_filters: 32
wandb: 	padding: same
wandb: 	type: bayes
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Found 9999 images belonging to 10 classes.
Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
selu
Epoch 1/5
 40/313 [==>...........................] - ETA: 2:24 - loss: 3.6660 - categorical_accuracy: 0.1000

In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
np.random.seed(2)
classes=["Amphibia","Animalia","Arachnida","Aves","Fungi","Insecta","Mammalia","Mollusca","Plantae","Reptilia"]
train_data,valid_data,test_data= get_data(path,True)
best_model=CNN(5,(3,3),0.001,1,'same','1',64,'elu',
              128,image_size,0.4,output_size,True,True)
best_model.model.compile(
optimizer=Adam(0.001),  # Optimizer
loss="categorical_crossentropy", metrics="categorical_accuracy")
  #train_data,valid_data,test_data=get_data(path,augmentation=True)
best_model.model.fit(train_data,epochs=1,batch_size=32,validation_data=valid_data)